# Preâmbulo

In [ ]:
import torch
import torch.nn as nn

# Word2Vec

A representação de palavras ou termos por *embeddings* é um dos conceitos mais fundamentais de Deep Learning em Processamento de Linguagem Naturais. O *word2vec*, proposto por Tomas Mikolov et al. no Google em 2013, foi um dos modelos iniciais utilizados para se aprender esse tipo de representação. Apesar de já ser considerado antigo, os conceitos desenvolvidos nas primeiras publicações ainda são úteis para o desenvolvimento de modelos mais avançados.

A ideia central do *word2vec* é a de que o significado de uma palavra está diretamente relacionado às palavras ao redor da mesma, ou seja, seu contexto. Por exemplo, podemos imaginar que as palavras que se encaixem em uma frase do tipo "hoje eu comi ___ no café da manhã" tenham uma certa proximidade de significado em alguns aspectos, e portanto possuam um grau de similaridade entre seus embeddings.

O artigo original propõe duas arquiteturas distintas para isso: **CBOW** (Continuous Bag-of-Words) e **Skip-Gram**.

<img width=600 src="https://www.researchgate.net/profile/Daniel-Braun-6/publication/326588219/figure/fig1/AS:652185784295425@1532504616288/Continuous-Bag-of-words-CBOW-CB-and-Skip-gram-SG-training-model-illustrations.png">

A arquitetura CBOW recebe como entrada da rede o conjunto de termos que formam o contexto e tenta prever o termo central da sequência. Já a arquitetura Skip-Gram tenta prever os termos do contexto com base na palavra central. Apesar dessa diferença, o objetivo em ambos os casos é gerar os embeddings dos termos. Portanto o que importa para nós no final são as representações aprendidas pelo modelo durante o treinamento e armazenados como parâmetros da rede.

Os detalhes da proposta inicial do algoritmo Word2Vec podem ser estudados pelo artigo no [link](https://research.google/pubs/pub41224/).

#Implementando

Vamos implementar um modelo word2vec utilizando a arquitetura CBOW.

Começamos definindo um texto simples para usarmos como dado, e gerando um vocabulário a partir disso.

In [ ]:
raw_text = """we are about to study the idea of a computational process
computational processes are abstract beings that inhabit computers
as they evolve processes manipulate other abstract things called data
the evolution of a process is directed by a pattern of rules
called a program people create programs to direct processes in effect
we conjure the spirits of the computer with our spells""".split()

# By deriving a set from `raw_text`, we deduplicate the array
vocab = set(raw_text)
vocab_size = len(vocab)

print("Total de %d tokens no vocabulário."% vocab_size)

Total de 44 tokens no vocabulário.


Podemos então definir dicionários que nos permitam encontrar um índice numérico para cada termo do vocabulário, assim como retornar o termo a partir do índice informado.

In [ ]:
word_to_ix = {word:ix for ix, word in enumerate(vocab)}
ix_to_word = {ix:word for ix, word in enumerate(vocab)}

print('\nÍndice da palavra "abstract" no dicionário:', word_to_ix["abstract"])
print('Palavra do índice 20 do dicionário:', ix_to_word[20])


Índice da palavra "abstract" no dicionário: 40
Palavra do índice 20 do dicionário: data


Com isso, conseguimos definir uma função que crie vetores (tensores) para um contexto contendo múltiplos termos. Isso é feito simplesmente listando os índices de cada palavra que faz parte do contexto dado como argumento.

In [ ]:
def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector("we are about to study".split(), word_to_ix)

tensor([ 2, 33, 23, 42,  0])

Definimos também os dados, pares contexto-alvo, percorrendo o texto tomando termo a termo como o alvo e as palavras ao redor como seu contexto. No caso, utilizamos uma janela de 2 termos em cada direção para delimitar o contexto.

In [ ]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right

data = []
for i in range(CONTEXT_SIZE, len(raw_text) - CONTEXT_SIZE):
    context = [raw_text[i-j] for j in range(CONTEXT_SIZE,0,-1)] + [raw_text[i+j] for j in range(1,CONTEXT_SIZE+1)]
    target = raw_text[i]
    data.append((context, target))

data[0]

(['we', 'are', 'to', 'study'], 'about')

Definimos então a estrutura da rede que utilizaremos. Tudo o que precisamos é de uma camada de Embedding, na qual aprenderemos as representações dos termos, seguida de algumas camadas lineares, as quais farão a previsão da palavra alvo.

In [ ]:
class CBOW(torch.nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(CBOW, self).__init__()

        self.embeddings = nn.Embedding(vocab_size, embedding_dim)

        self.linear1 = nn.Linear(embedding_dim, 128)
        self.activation_function1 = nn.ReLU()

        self.linear2 = nn.Linear(128, vocab_size)
        self.activation_function2 = nn.LogSoftmax(dim = -1)


    def forward(self, inputs):
        embeds = sum(self.embeddings(inputs)).view(1,-1)
        out = self.linear1(embeds)
        out = self.activation_function1(out)
        out = self.linear2(out)
        out = self.activation_function2(out)
        return out

    def get_word_emdedding(self, word):
        word = torch.tensor([word_to_ix[word]])
        return self.embeddings(word).view(1,-1)

Instaciamos e treinamos o modelo.

In [ ]:
EMDEDDING_DIM = 100

model = CBOW(vocab_size, EMDEDDING_DIM)
loss_function = nn.NLLLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001)

#TRAINING
for epoch in range(50):
    total_loss = 0

    for context, target in data:
        context_vector = make_context_vector(context, word_to_ix)

        log_probs = model(context_vector)

        total_loss += loss_function(log_probs, torch.tensor([word_to_ix[target]]))

    #optimize at the end of each epoch
    optimizer.zero_grad()
    total_loss.backward()
    optimizer.step()

Agora podemos testar a rede, fornecendo um contexto e verificando qual palavra o modelo prevê como termo central:

In [ ]:
#TESTING
context = ['people','create','to', 'direct']
context_vector = make_context_vector(context, word_to_ix)
a = model(context_vector)

#Print result
print(f'Raw text: \"{" ".join(raw_text)}\"\n')
print(f'Context: {context}\n')
print(f'Prediction: \'{ix_to_word[torch.argmax(a[0]).item()]}\'')

Raw text: "we are about to study the idea of a computational process computational processes are abstract beings that inhabit computers as they evolve processes manipulate other abstract things called data the evolution of a process is directed by a pattern of rules called a program people create programs to direct processes in effect we conjure the spirits of the computer with our spells"

Context: ['people', 'create', 'to', 'direct']

Prediction: 'programs'


Podemos também acessar os embeddings de termos individualmente:

In [ ]:
model.get_word_emdedding("programs")

tensor([[ 7.9654e-01,  7.6630e-01, -5.4276e-01,  1.8297e+00, -2.3810e+00,
         -9.5199e-01, -9.0614e-01,  4.3700e-01,  2.6124e-01,  2.3278e-01,
         -1.0762e+00,  6.4728e-03,  1.4939e+00,  1.0228e+00, -6.1191e-02,
          1.4968e+00, -1.7783e-01,  1.4811e+00, -1.4848e-02,  1.0777e+00,
         -1.0174e+00, -4.6248e-02,  1.0269e+00,  5.3987e-02, -1.2935e+00,
          1.3021e+00,  3.5443e-01,  1.6236e+00,  9.0056e-01,  6.1173e-01,
         -2.3818e+00, -7.6379e-01,  8.9388e-01, -4.4430e-01, -8.4214e-01,
          2.4058e+00, -1.9188e+00, -1.3686e-01, -1.0843e-01,  3.4437e-02,
          9.5590e-01, -2.3884e+00,  4.2155e-01, -8.6214e-01, -7.0116e-01,
         -1.3507e+00,  3.7163e-01, -1.6404e+00, -4.0019e-01, -1.9817e-02,
         -1.5885e-01,  7.7972e-01, -1.4686e+00, -5.1111e-01,  1.1729e-02,
         -1.1627e+00,  1.7932e-01, -5.0823e-01, -9.0858e-03,  1.5614e+00,
         -1.1414e+00, -1.1708e+00, -6.6843e-01, -7.6404e-02,  5.5218e-02,
         -2.1589e+00, -1.9049e-03, -1.